In [1]:
from pyspark import SparkConf
from pyspark import SparkContext

#conf = SparkConf()
#conf.set("spark.driver.extraClassPath", "/data/lectures/ttmda/mvit/jars/spark-csv_2.10-1.3.0.jar")
#conf.set("spark.executor.extraClassPath", "/data/lectures/ttmda/mvit/jars/spark-csv_2.10-1.3.0.jar")

sc = SparkContext()
print ( "Started spark version %s" % (sc.version) )

Started spark version 1.5.1


In [3]:
from pyspark.sql import Row

peopleTextRDD = sc.textFile("data/people.txt")

FIELD_SEPARATOR = ","
def fromPeopleLineToRow(line):
    cellsStrings = line.split( FIELD_SEPARATOR )
    name = cellsStrings[0].strip()
    age = int(cellsStrings[1])
    return Row(name, age)

peopleRowRDD = peopleTextRDD.map( fromPeopleLineToRow )

for row in peopleRowRDD.take(5):
    print( "Name: %s \t Age: %s" % (row[0], row[1]))

Name: Michael 	 Age: 29
Name: Andy 	 Age: 30
Name: Justin 	 Age: 19
Name: Michael 	 Age: 29


In [4]:
from pyspark.sql import HiveContext

sqlContext = HiveContext(sc)
peopleDataFrame = sqlContext.createDataFrame(peopleRowRDD, ["name", "age"]).cache()
peopleDataFrame.registerTempTable("people")
peopleDataFrame.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
|Michael| 29|
+-------+---+



In [5]:
sqlContext.sql("select count(*) from people").show()

+---+
|_c0|
+---+
|  4|
+---+



In [9]:
irisDf = sqlContext.read.format("com.databricks.spark.csv").\
                         option("header", "false").\
                         option("inferSchema", "true").\
                         load("data/iris.csv").\
                         cache()
irisDf.show()
irisDf.printSchema()
irisDf.registerTempTable("iris")

Py4JJavaError: An error occurred while calling o79.load.
: java.lang.ClassNotFoundException: Failed to load class for data source: com.databricks.spark.csv.
	at org.apache.spark.sql.execution.datasources.ResolvedDataSource$.lookupDataSource(ResolvedDataSource.scala:67)
	at org.apache.spark.sql.execution.datasources.ResolvedDataSource$.apply(ResolvedDataSource.scala:87)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:114)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:104)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:379)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.comman

In [6]:
from pyspark.sql import Row, functions as F
from pyspark.sql.window import Window

categories = sqlContext.sql(\
"""
    select 
      count(1) as quantity, 
      C4 as type, 
      1 as fake
    from 
      iris 
    group by C4
"""
    )\
    .select("type", "quantity",
             F.rowNumber()
               .over(Window
                   .partitionBy("fake")
                  )
             .alias("CatId")
    )\
    .cache()
print("Lazy")




Lazy


In [7]:
import datetime
start = datetime.datetime.now()
categories.show()
print(datetime.datetime.now() - start)

+---------------+--------+-----+
|           type|quantity|CatId|
+---------------+--------+-----+
|    Iris-setosa|      50|    1|
| Iris-virginica|      50|    2|
|Iris-versicolor|      50|    3|
+---------------+--------+-----+

0:00:04.157519


In [8]:
import datetime
start = datetime.datetime.now()
categories.show()
print(datetime.datetime.now() - start)

+---------------+--------+-----+
|           type|quantity|CatId|
+---------------+--------+-----+
|    Iris-setosa|      50|    1|
| Iris-virginica|      50|    2|
|Iris-versicolor|      50|    3|
+---------------+--------+-----+

0:00:00.906665


In [9]:
dataAndCategory = irisDf.join(categories, categories.type == irisDf.C4)
dataAndCategory.show()


# Save and load model
#model.save(sc, "myModelPath")
#sameModel = SVMModel.load(sc, "myModelPath")

+---+---+---+---+-----------+-----------+--------+-----+
| C0| C1| C2| C3|         C4|       type|quantity|CatId|
+---+---+---+---+-----------+-----------+--------+-----+
|5.1|3.5|1.4|0.2|Iris-setosa|Iris-setosa|      50|    1|
|4.9|3.0|1.4|0.2|Iris-setosa|Iris-setosa|      50|    1|
|4.7|3.2|1.3|0.2|Iris-setosa|Iris-setosa|      50|    1|
|4.6|3.1|1.5|0.2|Iris-setosa|Iris-setosa|      50|    1|
|5.0|3.6|1.4|0.2|Iris-setosa|Iris-setosa|      50|    1|
|5.4|3.9|1.7|0.4|Iris-setosa|Iris-setosa|      50|    1|
|4.6|3.4|1.4|0.3|Iris-setosa|Iris-setosa|      50|    1|
|5.0|3.4|1.5|0.2|Iris-setosa|Iris-setosa|      50|    1|
|4.4|2.9|1.4|0.2|Iris-setosa|Iris-setosa|      50|    1|
|4.9|3.1|1.5|0.1|Iris-setosa|Iris-setosa|      50|    1|
|5.4|3.7|1.5|0.2|Iris-setosa|Iris-setosa|      50|    1|
|4.8|3.4|1.6|0.2|Iris-setosa|Iris-setosa|      50|    1|
|4.8|3.0|1.4|0.1|Iris-setosa|Iris-setosa|      50|    1|
|4.3|3.0|1.1|0.1|Iris-setosa|Iris-setosa|      50|    1|
|5.8|4.0|1.2|0.2|Iris-setosa|Ir

In [10]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint


def parseData(row):
    data = [row['C0'], row['C1'], row['C2'], row['C3']]
    if row['CatId'] == 1 :
        lbl = 1
    else :
        lbl = 0
    return LabeledPoint(lbl, data)



parsedData = dataAndCategory.rdd.map(parseData)


# Build the model
model = SVMWithSGD.train(parsedData, iterations=100)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda pair: pair[0] != pair[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))


Training Error = 0.0
